In [1]:
import numpy as np
from pprint import pprint
import json

# Goals

Easy to define experiments at both the logical gate/pulse levels:
- parameterisation, custom waveforms, etc.
- timings, orderings

Simple syntax, access to existing ecosystem

Provide tooling for debugging, visualisation etc.

# Schedule
Container type of Operations and Resources

# Operation
An action
- gate
- pulse

# Resource
A device to perform an Operation on or with
- qubits
- pulsars

In [2]:
DEVICE_CFG = {
    "qubits":
    {
        "q0": {"mw_amp180": 0.5, "mw_motzoi": -0.25, "mw_duration": 20e-9,
               "mw_modulation_freq": 0, "mw_ef_amp180": 0.87, "mw_ch": "qcm0.s0",
               "ro_ch": "qrm0.s0", "ro_pulse_amp": 0.5, "ro_pulse_modulation_freq": 0,
               "ro_pulse_type": "square", "ro_pulse_duration": 150e-9,
               "ro_acq_delay": 120e-9, "ro_acq_integration_time": 100e-9,
               "ro_acq_weigth_type": "SSB",
               "init_duration": 250e-6
        },
        "q1": {"mw_amp180": 0.45, "mw_motzoi": -0.15, "mw_duration": 20e-9,
               "mw_modulation_freq": 0, "mw_ef_amp180": 0.27, "mw_ch": "qcm1.s0",
               "ro_ch": "qrm0.s1", "ro_pulse_amp": 0.5, "ro_pulse_modulation_freq": 0,
               "ro_pulse_type": "square", "ro_pulse_duration": 100e-9,
               "ro_acq_delay": 120e-9, "ro_acq_integration_time": 100e-9,
               "ro_acq_weigth_type": "SSB",
               "init_duration": 250e-6
        }
    },
    "edges":
    {
        "q0-q1": {
            "flux_duration": 40e-9,
            "flux_ch_control": "qcm0.s1", "flux_ch_target": "qcm1.s1",
            "flux_amp_control": 0.5,  "flux_amp_target": 0,
            "phase_correction_control": 0,
            "phase_correction_target": 0}
    }
}

In [3]:
from quantify.sequencer.types import Schedule
from quantify.sequencer.gate_library import X, X90, Measure
from quantify.sequencer.pulse_library import SquarePulse
from quantify.sequencer.backends.pulsar_backend import pulsar_assembler_backend
from quantify.sequencer.resources import QubitResource, Pulsar_QCM_sequencer, Pulsar_QRM_sequencer

ModuleNotFoundError: No module named 'quantify.sequencer'

In [4]:
sched = Schedule("Chevron Experiment")
q0, q1 = (QubitResource("q0"), QubitResource("q1"))

for amp in np.linspace(0.2, 0.8, 3):
    for duration in np.arange(20e-9, 100e-9, 20e-9):
        begin = sched.add(X(q0.name))
        sched.add(X(q1.name), ref_op=begin, ref_pt='start')
        square = sched.add(SquarePulse(amp=amp, duration=duration, ch='qcm1.s0'))
        sched.add(X90(q1.name), ref_op=square)
        sched.add(X90(q0.name), ref_op=square)
        sched.add(Measure(q0.name, q1.name))

NameError: name 'Schedule' is not defined

In [5]:
qcm0_s0 = Pulsar_QCM_sequencer('qcm0.s0', instrument_name='qcm0', seq_idx=0)
qcm1_s0 = Pulsar_QCM_sequencer('qcm1.s0', instrument_name='qcm1', seq_idx=0)
qrm0_s0 = Pulsar_QRM_sequencer('qrm0.s0', instrument_name='qrm0', seq_idx=0)
qrm0_s1 = Pulsar_QRM_sequencer('qrm0.s1', instrument_name='qrm0', seq_idx=1)

sched.add_resources([q0, q1, qcm0_s0, qrm0_s0, qcm1_s0, qrm0_s1])

NameError: name 'Pulsar_QCM_sequencer' is not defined

In [6]:
from quantify.sequencer.compilation import qcompile

sched, cfg = qcompile(sched, DEVICE_CFG, backend=pulsar_assembler_backend)

ModuleNotFoundError: No module named 'quantify.sequencer'

In [7]:
sched

NameError: name 'sched' is not defined

In [8]:
cfg

NameError: name 'cfg' is not defined

In [9]:
from quantify.sequencer.backends.visualization import pulse_diagram_plotly

pulse_diagram_plotly(sched, ch_list=['qcm0.s0', 'qcm1.s0', 'qrm0.s0'])

ModuleNotFoundError: No module named 'quantify.sequencer'

In [10]:
with open(cfg['qcm0.s0'], 'r') as f:
    program = json.load(f)
pprint(program['program'])

NameError: name 'cfg' is not defined

In [11]:
pprint(program['waveforms']['awg'])

NameError: name 'program' is not defined




Gates compiled into pulses

Pulses ordered by timing constraints

Pulses associated to resource

Resources joined to create a channel

QASM generated for each channel


